# Energy Chain 1
## A proof of concept

### The objective
As I have discussed in the following article.... blockchain holds some promise as a technology that might be able to simplify the energy system through the creation of a simplified, decentralised trading mechanism that isn't subject the whims of a centralised system operator such as AEMO. By leveraging distributed ledger technology, participants in the energy market could transact directly with one another, increasing transparency and reducing the need for intermediaries. This approach has the potential to streamline settlement processes, lower transaction costs, and empower both producers and consumers to manage energy flows more efficiently. Most importantly, I have a belief that it will lower energy costs, the #1 object of Australia's National Energy Objective.

Who knows, I might be totally wrong. But in my mind, through appropriate incentivisation, this is the future. Regardless, it's going to be a fun experiment, so let's get into it. 

Funnily enough my thesis from business school was the application of blockchain in the pharmaceutical industry. I only got a 2:1 for my thesis, so it's time for some redemption!

## The Architecture

You can use frameworks such as hyperledger fabric, and blockchain-as-a-service such as IBM Blockchain. But for the sake of this proof of concept, let's get back to basics...using pure Python Code. This will give us the best idea of how blockchain fundamentally works.

## Development

Let's begin by setting up the dependencies:

In [3]:
import time # provides time related functions such as getting the current time
import json # provides functions for working with JSON data
import hashlib # provides functions for hashing data
from dataclasses import dataclass, field # provides a decorator and functions for creating data classes. Dataclasses help to reduce boilerplate code when creating classes that primarily store data.
from typing import List, Optional, Dict, Tuple # provides type hints for various data structures, type hints help improve code readability and enable static type checking that can catch potential errors before runtime.
from ecdsa import SigningKey, SECP256k1, VerifyingKey, BadSignatureError # provides functions for ECDSA cryptography, ECDSA is a widely used public key cryptography algorithm based on elliptic curves.

We now want to set up two utilities:

- `sha256(data: str) -> str`: This function takes a string input and returns its SHA-256 hash. Hashing is fundamental in blockchain for creating unique identifiers for blocks and transactions.
- `serialize(obj) -> str`: This function converts a Python object into a deterministic JSON string. By sorting keys and using consistent separators, it ensures that the same object always produces the same string, which is important for reliable hashing and verification.

In [4]:

def sha256(data: str) -> str:
    return hashlib.sha256(data.encode("utf-8")).hexdigest()

def serialize(obj) -> str:
    # Deterministic JSON for hashing
    return json.dumps(obj, sort_keys=True, separators=(",", ":"))

### Setting up the wallets

Each user will own a wallet that stores cryptographic keys (private and public keys) used to sign and verify transactions. A wallet allows a user to send, receive, and manage digital assets securely.

`class Wallet:` defines a wallet object for blockchain users.

`__init__(self)`:
`self.sk = SigningKey.generate(curve=SECP256k1)` creates a new private key using the SECP256k1 curve called a signing key.
`self.vk = self.sk.get_verifying_key()` gets the corresponding public key called a verifying key. 
`self.address = sha256(self.vk.to_string().hex())` creates a wallet address by hashing the public key.


`sign(self, message: str) -> str:`
``signature = self.sk.sign(message.encode("utf-8")) return signature.hex()` Signs a message with the private key and returns the signature as a hex string.

`verifier(self) -> VerifyingKey:` Returns the public key for verifying signatures.
` return self.vk`


In [5]:
class Wallet:
    def __init__(self):
        self.sk = SigningKey.generate(curve=SECP256k1)
        self.vk = self.sk.get_verifying_key()
        # Address = hash of compressed public key
        self.address = sha256(self.vk.to_string().hex())

    def sign(self, message: str) -> str:
        signature = self.sk.sign(message.encode("utf-8"))
        return signature.hex()

    def verifier(self) -> VerifyingKey:
        return self.vk

Now let's run through an example of this code in action

In [6]:
# Create two wallets
alice_wallet = Wallet()
bob_wallet = Wallet()

# Alice wants to send a message to Bob
message = "Alice pays Bob 10 tokens"

# Alice signs the message with her wallet
signature = alice_wallet.sign(message)
print(f"Signature: {signature}")

# Bob (or anyone) can verify Alice's signature using her public key
try:
    alice_wallet.verifier().verify(bytes.fromhex(signature), message.encode("utf-8"))
    print("Signature is valid!")
except Exception:
    print("Signature is invalid!")

Signature: a7a2d8738ce1147b06d8db5c59c7c2d4e8ee06e0d923b76e37f1200b7513deb502bf592463a5eea143fc05bb709cab02e472ace4e54342ad3d6567b28e21df31
Signature is valid!


Now let's simplify the verification with another utility

In [7]:
def verify_signature(vk: VerifyingKey, message: str, signature_hex: str) -> bool:
    try:
        vk.verify(bytes.fromhex(signature_hex), message.encode("utf-8"))
        return True
    except BadSignatureError:
        return False

In [8]:
# Bob verifies the signature
is_valid = verify_signature(alice_wallet.verifier(), message, signature)
print(is_valid) 

True


### Transactions and market logic

Since we have now set up how participants will send and verify messages, let's set up the market logic. 

- `class Transaction:` defines a blockchain transaction using a data class that makes storing data simpler.
- `tx_type: str` specifies the type of transaction (e.g., `"offer"` or `"trade"`).
- `sender: str` is the address of the user initiating the transaction.
- `payload: Dict` holds transaction details, such as `{'kwh': 10, 'price_per_kwh': 2.5, 'offer_id': '...'}`.
- `signature: Optional[str] = None` stores the digital signature for the transaction (if present). This is optional to allow you to create the transaction before signing it!

**Methods:**
- `to_dict(self) -> Dict`: returns a dictionary representation of the transaction, including all fields.
- `msg(self) -> str`: returns a deterministic JSON string of the transaction (excluding the signature), used for signing.
- `txid(self) -> str`: returns a SHA-256 hash of the serialized transaction dictionary, serving as a unique transaction ID.

In [9]:
@dataclass
class Transaction:
    tx_type: str  # "offer" or "trade"
    sender: str   # address initiating
    payload: Dict # e.g., {'kwh': 10, 'price_per_kwh': 2.5, 'offer_id': '...'}
    signature: Optional[str] = None

    def to_dict(self) -> Dict:
        return {
            "tx_type": self.tx_type,
            "sender": self.sender,
            "payload": self.payload,
            "signature": self.signature
        }

    def msg(self) -> str:
        # Message to sign excludes signature
        return serialize({
            "tx_type": self.tx_type,
            "sender": self.sender,
            "payload": self.payload
        })

    def txid(self) -> str:
        return sha256(serialize(self.to_dict()))

In [10]:
# Example transaction: Alice offers to sell 10 kWh at 2.5 tokens/kWh

# Create an offer transaction
tx = Transaction(
    tx_type="offer",
    sender=alice_wallet.address,
    payload={"kwh": 10, "price_per_kwh": 2.5}
)
print(f"Transaction before signing: {tx}")
print(f"This is why we need the to_dict method to return this: {tx.to_dict()}")

# Alice signs the transaction
tx.signature = alice_wallet.sign(tx.msg())
print(f"Signed Transaction: {tx.to_dict()}")

# Transaction ID - the unique identifier for this transaction
print(f"Transaction ID: {tx.txid()}")


# Bob verifies the transaction signature
is_tx_valid = verify_signature(
    alice_wallet.verifier(),
    tx.msg(),
    tx.signature
)
print(f"Is transaction valid? {is_tx_valid}")




Transaction before signing: Transaction(tx_type='offer', sender='dee75507052dc8a1ddd0105a9011b38b22238291da5e590386fed6cca826fb33', payload={'kwh': 10, 'price_per_kwh': 2.5}, signature=None)
This is why we need the to_dict method to return this: {'tx_type': 'offer', 'sender': 'dee75507052dc8a1ddd0105a9011b38b22238291da5e590386fed6cca826fb33', 'payload': {'kwh': 10, 'price_per_kwh': 2.5}, 'signature': None}
Signed Transaction: {'tx_type': 'offer', 'sender': 'dee75507052dc8a1ddd0105a9011b38b22238291da5e590386fed6cca826fb33', 'payload': {'kwh': 10, 'price_per_kwh': 2.5}, 'signature': '346ff401e22c26be060d9bfceabbaeab8d255abb3753b270bc7f39658a2e83946513d1b442f63bb3ac6532de7f6591416dacd5a5b0c6db0e566725910de8e590'}
Transaction ID: 24f635986fa9d5ea04e41d5ba71921df7c43d21c72c95e1609a308560628c99d
Is transaction valid? True


### The Block

Cool right? Now we need to set up two set of data, the blocks, and the bockchain. Let's start with the smallest bit, the block.

- `@dataclass` automatically generates methods like `__init__` for the `Block` class.
- `index: int` is the block's position in the blockchain.
- `prev_hash: str` stores the hash of the previous block, linking blocks together.
- `timestamp: float` records when the block was created.
- `transactions: List[Transaction]` holds all transactions in the block.

**Methods:**
- `header(self) -> Dict`: returns a dictionary with block metadata and a root hash of all transaction IDs (`tx_root`).
- `block_hash(self) -> str`: returns a SHA-256 hash of the block's header and transaction IDs, serving as the block's unique identifier.


In [11]:
@dataclass
class Block:
    index: int
    prev_hash: str
    timestamp: float
    transactions: List[Transaction] = field(default_factory=list)

    def header(self) -> Dict:
        return {
            "index": self.index,
            "prev_hash": self.prev_hash,
            "timestamp": self.timestamp,
            "tx_root": sha256(serialize([tx.txid() for tx in self.transactions]))
        }

    def block_hash(self) -> str:
        return sha256(serialize({"header": self.header(), "txs": [tx.txid() for tx in self.transactions]}))

In [12]:
# Assume you already have some transactions
transactions = [tx]  # tx from your previous example

# Create a block
block = Block(
    index=0,
    prev_hash="0" * 64,  # Genesis block or previous block's hash
    timestamp=time.time(),
    transactions=transactions
)

print("Block header:", block.header())
print("Block hash:", block.block_hash())

Block header: {'index': 0, 'prev_hash': '0000000000000000000000000000000000000000000000000000000000000000', 'timestamp': 1764223140.861375, 'tx_root': 'a856124d544af6efc420b061e8ae353701cae3936cfc8be3cce255ff5feff618'}
Block hash: ad7d44359ec09abf42e28225e695d9e29ad4c1f9898ed6430e500abc7b6979e2


### The Blockchain


We begin by setting up the blockchain instant `class Blockchain` and setting up the initial values:

```python 
def __init__(self):
        self.chain: List[Block] = [] # This list of blocks in the blockchain
        self.balances: Dict[str, float] = {}   # tracks token balances for each wallet address
        self.energy: Dict[str, float] = {}     # tracks energy (kWh holdings) for each wallet address
        self.offers: Dict[str, Dict] = {}      # store active offers
        self.verifiers: Dict[str, VerifyingKey] = {}  # maps wallet addresses to their public key
        self.genesis() # creates and adds the first block to the chain
```

Next we need to allow the ability to add a new wallet to the blockchain system and set its initial balance:

```python
def register_wallet(self, wallet: Wallet, initial_tokens: float = 0.0, initial_kwh: float = 0.0):
        self.verifiers[wallet.address] = wallet.verifier()
        self.balances.setdefault(wallet.address, 0.0)
        self.energy.setdefault(wallet.address, 0.0)
        self.balances[wallet.address] += initial_tokens
        self.energy[wallet.address] += initial_kwh
```

Next we need to creates the genesis block:

```python
  def genesis(self):
        genesis_block = Block(index=0, prev_hash="0"*64, timestamp=time.time(), transactions=[])
        self.chain.append(genesis_block)
```

Next we set up the ability to add a block to the blockchain, this allows participants to submit transactions:

```python
 def add_block(self, block: Block) -> bool:
        # Basic block link check
        if block.prev_hash != self.chain[-1].block_hash():
            return False
        # Validate transactions against to ensure participants can only submit transactions that are legitimate
        for tx in block.transactions:
            if not self.validate_transaction(tx):
                return False
            self.apply_transaction(tx)
        self.chain.append(block)
        return True
```
...and we now need to set up that validation method

```python
def validate_transaction(self, tx: Transaction) -> bool:
        # Signature verification
        if tx.sender not in self.verifiers:
            return False
        vk = self.verifiers[tx.sender]
        if not tx.signature or not verify_signature(vk, tx.msg(), tx.signature):
            return False

        if tx.tx_type == "offer":
            # Check the sender has the energy they are offering and that price is positive
            kwh = tx.payload.get("kwh", 0.0)
            price = tx.payload.get("price_per_kwh", 0.0)
            offer_id = tx.payload.get("offer_id")
            if not offer_id or kwh <= 0 or price <= 0:
                return False
            if self.energy.get(tx.sender, 0.0) < kwh:
                return False
            # Prevent duplicate offer ids
            if offer_id in self.offers:
                return False
            return True

        elif tx.tx_type == "trade":
            # Buyer accepts an offer: payload needs offer_id and buyer field
            offer_id = tx.payload.get("offer_id")
            buyer = tx.payload.get("buyer")
            if not offer_id or not buyer:
                return False
            # Signature must be from buyer
            if tx.sender != buyer:
                return False
            # Offer must exist
            offer = self.offers.get(offer_id)
            if not offer:
                return False
            seller = offer["seller"]
            kwh = offer["kwh"]
            price = offer["price_per_kwh"]
            total_cost = kwh * price
            # Buyer must have enough tokens; seller must still have energy (not double-spent)
            if self.balances.get(buyer, 0.0) < total_cost:
                return False
            if self.energy.get(seller, 0.0) < kwh:
                return False
            return True

        return False
```

...getting there, its not too bad once you break it down step by step. Finally, we need to add the ability to apply a transaction to the blockchain, which is essentially to update the blockchains state based on the transaction type. 

For "offer" transactions: It adds a new energy offer to the self.offers dictionary, recording the seller, amount of energy (kWh), and price.

For "trade" transactions:
It removes (consumes) the offer, transfers tokens from buyer to seller, and transfers energy from seller to buyer, updating their balances and energy holdings.

```python
 def apply_transaction(self, tx: Transaction):
        if tx.tx_type == "offer":
            offer_id = tx.payload["offer_id"]
            self.offers[offer_id] = {
                "seller": tx.sender,
                "kwh": tx.payload["kwh"],
                "price_per_kwh": tx.payload["price_per_kwh"]
            }
        elif tx.tx_type == "trade":
            offer_id = tx.payload["offer_id"]
            offer = self.offers.pop(offer_id)  # consume the offer
            seller = offer["seller"]
            buyer = tx.payload["buyer"]
            kwh = offer["kwh"]
            price = offer["price_per_kwh"]
            total_cost = kwh * price
            # Transfer tokens and energy
            self.balances[buyer] -= total_cost
            self.balances[seller] += total_cost
            self.energy[seller] -= kwh
            self.energy[buyer] += kwh
```



Full code:

In [13]:
class Blockchain:
    def __init__(self, difficulty: int = 2):
        self.chain: List[Block] = []
        self.balances: Dict[str, float] = {}   # token balances
        self.energy: Dict[str, float] = {}     # kWh holdings
        self.offers: Dict[str, Dict] = {}      # offer_id -> offer payload
        self.verifiers: Dict[str, VerifyingKey] = {}  # address -> VK
        self.genesis()

    def register_wallet(self, wallet: Wallet, initial_tokens: float = 0.0, initial_kwh: float = 0.0):
        self.verifiers[wallet.address] = wallet.verifier()
        self.balances.setdefault(wallet.address, 0.0)
        self.energy.setdefault(wallet.address, 0.0)
        self.balances[wallet.address] += initial_tokens
        self.energy[wallet.address] += initial_kwh

    def genesis(self):
        genesis_block = Block(index=0, prev_hash="0"*64, timestamp=time.time(), transactions=[])
        self.chain.append(genesis_block)

    def add_block(self, block: Block) -> bool:
        # Basic block link check
        if block.prev_hash != self.chain[-1].block_hash():
            return False
        # Validate transactions against current state
        for tx in block.transactions:
            if not self.validate_transaction(tx):
                return False
            self.apply_transaction(tx)
        self.chain.append(block)
        return True

    def validate_transaction(self, tx: Transaction) -> bool:
        # Signature verification
        if tx.sender not in self.verifiers:
            return False
        vk = self.verifiers[tx.sender]
        if not tx.signature or not verify_signature(vk, tx.msg(), tx.signature):
            return False

        if tx.tx_type == "offer":
            # Check the sender has the energy they are offering and that price is positive
            kwh = tx.payload.get("kwh", 0.0)
            price = tx.payload.get("price_per_kwh", 0.0)
            offer_id = tx.payload.get("offer_id")
            if not offer_id or kwh <= 0 or price <= 0:
                return False
            if self.energy.get(tx.sender, 0.0) < kwh:
                return False
            # Prevent duplicate offer ids
            if offer_id in self.offers:
                return False
            return True

        elif tx.tx_type == "trade":
            # Buyer accepts an offer: payload needs offer_id and buyer field
            offer_id = tx.payload.get("offer_id")
            buyer = tx.payload.get("buyer")
            if not offer_id or not buyer:
                return False
            # Signature must be from buyer
            if tx.sender != buyer:
                return False
            # Offer must exist
            offer = self.offers.get(offer_id)
            if not offer:
                return False
            seller = offer["seller"]
            kwh = offer["kwh"]
            price = offer["price_per_kwh"]
            total_cost = kwh * price
            # Buyer must have enough tokens; seller must still have energy (not double-spent)
            if self.balances.get(buyer, 0.0) < total_cost:
                return False
            if self.energy.get(seller, 0.0) < kwh:
                return False
            return True

        return False

    def apply_transaction(self, tx: Transaction):
        if tx.tx_type == "offer":
            offer_id = tx.payload["offer_id"]
            self.offers[offer_id] = {
                "seller": tx.sender,
                "kwh": tx.payload["kwh"],
                "price_per_kwh": tx.payload["price_per_kwh"]
            }
        elif tx.tx_type == "trade":
            offer_id = tx.payload["offer_id"]
            offer = self.offers.pop(offer_id)  # consume the offer
            seller = offer["seller"]
            buyer = tx.payload["buyer"]
            kwh = offer["kwh"]
            price = offer["price_per_kwh"]
            total_cost = kwh * price
            # Transfer tokens and energy
            self.balances[buyer] -= total_cost
            self.balances[seller] += total_cost
            self.energy[seller] -= kwh
            self.energy[buyer] += kwh


So let's try this bad boy out!



In [ ]:
chain = Blockchain()

print(chain)


9fc04262f3abb5c732fa8382d97356e1fbd6273296c11a37b545ffcd97c905e8
9f32523a7092f16a30778886f67c6390a1481d6809ca12da42be3088e1dda6b9
{'9fc04262f3abb5c732fa8382d97356e1fbd6273296c11a37b545ffcd97c905e8': 0.0, '9f32523a7092f16a30778886f67c6390a1481d6809ca12da42be3088e1dda6b9': 50.0}
{'9fc04262f3abb5c732fa8382d97356e1fbd6273296c11a37b545ffcd97c905e8': 20.0, '9f32523a7092f16a30778886f67c6390a1481d6809ca12da42be3088e1dda6b9': 0.0}


In [ ]:
# Create wallets
prosumer_a = Wallet()
prosumer_b = Wallet()

print(prosumer_a.address)
print(prosumer_b.address)

In [ ]:
# Register wallets with initial balances
chain.register_wallet(prosumer_a, initial_tokens=0.0, initial_kwh=20.0)   # A has energy to sell
chain.register_wallet(prosumer_b, initial_tokens=50.0, initial_kwh=0.0)   # B has tokens to buy

print(chain.balances)
print(chain.energy)

In [ ]:
# Prosumer A creates an energy offer
offer_id = "offer-002"

offer_tx = Transaction(
    tx_type="offer",
    sender=prosumer_a.address,
    payload={"offer_id": offer_id, "kwh": 5.0, "price_per_kwh": 2.0}
)

offer_tx.signature = prosumer_a.sign(offer_tx.msg())

offer_block = Block(
    index=len(chain.chain),
    prev_hash=chain.chain[-1].block_hash(),
    timestamp=time.time(),
    transactions=[offer_tx]
)
assert chain.add_block(offer_block)

print("After offer:")
print(chain.offers)


After offer:
{'offer-001': {'seller': '9fc04262f3abb5c732fa8382d97356e1fbd6273296c11a37b545ffcd97c905e8', 'kwh': 5.0, 'price_per_kwh': 2.0}, 'offer-002': {'seller': '9fc04262f3abb5c732fa8382d97356e1fbd6273296c11a37b545ffcd97c905e8', 'kwh': 5.0, 'price_per_kwh': 2.0}}


So we now have an offer ready to be snapped up by Prosumer 2.

In [21]:
# Prosumer B accepts the offer
trade_tx = Transaction(
    tx_type="trade",
    sender=prosumer_b.address,  # buyer must be the sender
    payload={"offer_id": offer_id, "buyer": prosumer_b.address}
)
trade_tx.signature = prosumer_b.sign(trade_tx.msg())

trade_block = Block(
    index=len(chain.chain),
    prev_hash=chain.chain[-1].block_hash(),
    timestamp=time.time(),
    transactions=[trade_tx]
)
assert chain.add_block(trade_block)
print("After trade:")
print(chain.balances)
print(chain.energy)
print(chain.offers)

After trade:
{'9fc04262f3abb5c732fa8382d97356e1fbd6273296c11a37b545ffcd97c905e8': 10.0, '9f32523a7092f16a30778886f67c6390a1481d6809ca12da42be3088e1dda6b9': 40.0}
{'9fc04262f3abb5c732fa8382d97356e1fbd6273296c11a37b545ffcd97c905e8': 15.0, '9f32523a7092f16a30778886f67c6390a1481d6809ca12da42be3088e1dda6b9': 5.0}
{'offer-001': {'seller': '9fc04262f3abb5c732fa8382d97356e1fbd6273296c11a37b545ffcd97c905e8', 'kwh': 5.0, 'price_per_kwh': 2.0}}


In [ ]:
# Inspect chain integrity
for b in chain.chain:
    print(b.index, b.prev_hash[:8], b.block_hash()[:8], len(b.transactions))


0 00000000 b1c1034b 0
1 b1c1034b 8c617df8 1
2 8c617df8 c63c9135 1
3 c63c9135 d96a51e5 1


How awesome is that, we have set up a basic blockchain with basic encryption and energy trading between two prosumers (peer-to-peer). It goes to show how building something makes it far easier to undersand, than just living in the theoretical world. 

This demonstrates how blockchain technology can securely automate transactions, track balances, and enable direct energy exchange—without a central authority. We now have a working foundation for decentralized energy markets in Python!
